In [ ]:
# importing Qiskit
from qiskit import (
	IBMQ,
	QuantumCircuit,
	transpile,
)
from qiskit.providers import Backend
from qiskit.providers.aer.noise import NoiseModel

import statistics
import numpy as np
from matplotlib.pyplot import figure
import math
import datetime
from IPython.display import display
import os
import time
import pickle

In [ ]:
provider = IBMQ.load_account()

In [ ]:
hw_backend = provider.get_backend("ibmq_belem")

In [ ]:
def create_test_circs(backend):
		qubits = backend.configuration().n_qubits
		circs = []
		circs_params = []
		samples = np.linspace(0, 250, 15, dtype=int)[1:]*2
		for i in range(qubits):
			for cnt in samples:
				qc = QuantumCircuit(qubits, qubits)
				for j in range(cnt):
					qc.x(i)
					qc.barrier()
				qc.measure_all(add_bits=False)
				circs.append(qc)
				circs_params.append({"expected result": {"0"*qubits : 1},
									 "desc": f"{cnt} chained not gates on qubit #{i}"})


		for cnt in samples:
			qc = QuantumCircuit(qubits, qubits)
			for j in range(cnt):
				qc.x(range(qubits))
				qc.barrier()
			qc.measure_all(add_bits=False)
			circs.append(qc)
			circs_params.append({"expected result": {"0"*qubits if cnt % 2 == 0 else "1"*qubits : 1},
									"desc": f"{cnt} chained not gates on all qubits"})

		for cnt in samples:
			qc = QuantumCircuit(qubits, qubits)
			for j in range(cnt):
				qc.id(range(qubits))
				qc.barrier()
			qc.measure_all(add_bits=False)
			circs.append(qc)
			circs_params.append({"expected result": {"0"*qubits : 1},
									"desc": f"{cnt} chained id gates (delay of gates) on all qubits"})
		
		qc = QuantumCircuit(qubits, qubits)
		qc.barrier()
		qc.measure_all(add_bits=False)
		circs.append(qc)
		circs_params.append({"expected result": {"0"*qubits : 1},
								"desc": f"setup and measure without any gates"})

		return transpile(circs, backend, optimization_level=0, seed_transpiler=42), circs_params

In [ ]:
class Measurement:
	def __init__(self, backend, circs, params):
		self.backend = backend
		self.properties_before = None
		self.properties_before_ts = None
		self.properties_after = None
		self.properties_after_ts = None
		self.properties_runtime = None
		self.properties_runtime_ts = None
		self.result = None
		self.job = None
		self.circuits = circs
		self.circuit_params = params
		self.noise_model = None

	def run(self):
		self.properties_before_ts = datetime.datetime.now()
		self.properties_before = self.backend.properties()

		self.job = self.backend.run(self.circuits, shots=10000)
		self.result = self.job.result().get_counts()
		self.noise_model = NoiseModel.from_backend(self.backend)

		self.properties_runtime_ts = self.job.time_per_step()["RUNNING"]
		self.properties_runtime = self.backend.properties(datetime=self.properties_runtime_ts)

		self.properties_after = self.backend.properties()
		self.properties_after_ts = datetime.datetime.now()


In [ ]:
test_circs, test_params = create_test_circs(hw_backend)

In [ ]:
out_folder = "output_long_experiment"
if not os.path.exists(out_folder):
	os.mkdir(out_folder)

In [ ]:
total_runtime = 48 #in hours
interval = 30 #in minutes
steps = total_runtime*60 // interval
start = datetime.datetime.now()
try:
	for i in range(steps):
		measure = Measurement(hw_backend, test_circs, test_params)
		measure.run()
		with open(os.path.join(out_folder, f"measurement_{i}"), "wb") as file:
			pickle.dump(measure, file)

		time.sleep(interval*60)
except KeyboardInterrupt as e:
	print(f"stopped after {i} iterations")
	print(f"started {str(start)}")
	print(f"end {str(datetime.datetime.now())}")
